In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.utils import save_image
from torch.autograd import Variable
from tqdm import tqdm

In [ ]:
#-------basic GAN--------
'''
####model architecture####
-- define generator
-- define discriminator
-- adversarial loss
-- gen optimizer
-- discr optimizer

####training loop####
##genrator training##
-- generate noise vector
-- noise vector --> generator --> fake output
-- gen loss --> adv loss --> disc (fake, real)
-- gen optimizer steps
##discriminator training##
-- real loss --> adv loss --> real images
-- fake loss --> adv loss --> fake/generator images
-- final loss --> (real loss + fake loss)/2
-- dicriminator optimizer step
'''



In [ ]:
device = 'cuda'
batch_size = 64
epochs = 200
z_dim=100
sample_interval = 100 #400
LR = 0.0002
B1 = 0.5
B2 = 0.999
IMG_SIZE = 32
CHANNELS = 3

# Dataloader

In [ ]:
!mkdir mnist

In [ ]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(IMG_SIZE), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
'''
img_shape = None
for img,_ in dataloader_1:
    img_shape = img.shape
    break

img_shape = np.array(img_shape)
int(np.prod(img_shape[1:]))
'''

# Generator and Discriminator

In [ ]:
def noise_vector(batch_size, dim):
    return torch.randn(batch_size, dim, device=device)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.init_size = IMG_SIZE // 4
        self.linear = nn.Sequential(nn.Linear(z_dim, 128 * self.init_size ** 2))
        self.conv = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, CHANNELS, 3, stride=1, padding=1),
            nn.Tanh(),
        )  
        
    def forward(self, noise):
        img = self.linear(noise)
        img = img.view(img.size(0), 128, self.init_size, self.init_size)
        out = self.conv(img)
        #print(out.shape)
        return out
        
    

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(CHANNELS, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128)
        )

        # The height and width of downsampled image
        ds_size = IMG_SIZE // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

        
    def forward(self, img):
        out = self.model(img)
        #print(out.shape)
        out = out.view(out.shape[0], -1)
        #print(out.shape)
        output = self.adv_layer(out)

        return output

In [ ]:
adv_loss = nn.BCELoss()

gen = Generator()
dis = Discriminator()

gen.cuda()
dis.cuda()
adv_loss.cuda()

gen_opt = torch.optim.Adam(gen.parameters(), lr=LR, betas=(B1, B2))
dis_opt = torch.optim.Adam(dis.parameters(), lr=LR, betas=(B1, B2))



# Training

In [ ]:
!mkdir images

In [ ]:
Tensor = torch.cuda.FloatTensor

for epoch in tqdm(range(epochs)):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        gen_opt.zero_grad()

        # Sample noise as generator input
        noise = Variable(noise_vector(imgs.shape[0], z_dim))
        
        
        # Generate a batch of images
        gen_imgs = gen(noise)
        #print(gen_imgs.shape)
        dis_out = dis(gen_imgs)
        #print(dis_out.shape)
        
        # Loss measures generator's ability to fool the discriminator
        g_loss = adv_loss(dis(gen_imgs), valid)

        g_loss.backward()
        gen_opt.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        
        dis_opt.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adv_loss(dis(real_imgs), valid)
        fake_loss = adv_loss(dis(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        dis_opt.step()
    
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
    '''
    if batches_done % 20 == 0:
        print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

    '''
    
        

In [ ]:
!python -m zipfile -c images.zip ./images

In [ ]:
from IPython.display import FileLink
FileLink(r'images.zip')
